In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


import pandas as pd 
application_df = pd.read_csv("csv")
application_df.head()

In [ ]:
df3 = application_df.drop([''], axis=1)
df3.head()

In [ ]:
df3.shape

In [ ]:
dummiesdf3 = pd.get_dummies(df3, columns=['column name'], drop_first=True, dtype=float)
dummiesdf3.shape

In [ ]:
dummiesdfx3 = dummiesdf3.drop('column', axis=1)
dummiesdfx3.head()

In [ ]:
dummiesdfy3 = dummiesdf3['column']
dummiesdfy3

In [ ]:
X3 = dummiesdfx3
y3 = dummiesdfy3

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, random_state = 50)

In [ ]:
scaler = StandardScaler()

X3_scaler = scaler.fit(X3_train)

X3_train_scaled = X3_scaler.transform(X3_train)
X3_test_scaled = X3_scaler.transform(X3_test)

In [ ]:
def create_model(hp):
    nn3 = tf.keras.models.Sequential()

    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    nn3.add(tf.keras.layers.Dense(units=hp.Int('first_units',
            min_value=1,
            max_value=50,
            step=2
            ), activation=activation, input_dim=28))
    
    for i in range(hp.Int('num_layers', 1, 6)):
        nn3.add(tf.keras.layers.Dense(units=hp.Int('units_'+str(i),
            min_value=1,
            max_value=50,
            step=2),
            activation=activation))
    
    nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    nn3.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn3

In [ ]:
import kerastuner as kt
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
tuner.search(X3_train_scaled,y3_train,epochs=50,validation_data=(X3_test_scaled,y3_test))

In [ ]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X3_test_scaled,y3_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")